Fitting with user-defined functions
===

There are numerous ways to express functions in ROOT (https://root.cern.ch/doc/master/classTF1.html) in this example we'll explore a general method of defining fit functions.  Again we will use the ROOT fitting framework for the next couple of exercies to illustrate some general featues of the non-linear fitting problems.  Some of the examples we see may be replicated in scipy.optimize, but not neceassarily all of them.

ROOT classes used here:
* [TF1](https://root.cern.ch/doc/master/classTF1.html): 1D function
* [TH1F](https://root.cern.ch/doc/master/classTH1.html): 1D histogram (the content of each bin is described by a float)

In [ ]:
import ROOT as r
%jsroot off

: 

First we define a function and store it in a TF1 object.  In this example we will use Pythonic interfaces, next time we'll see how the same patterns can be used in C++.

In [ ]:
from math import pow, exp

# A function producing two peaks on top of an exponentialy 
# falling background.  Depends on several parameters.
# Generic interface for fcn of n input-values and m parameters
# Functions with this interface may be used to construct a "TFunction" or TF1
# vx is the independent value(s): array like
# p is the parameter value(s): array like

def myfunction(vx, par):
    x=vx[0]
    # background model
    bkgScale=par[0]
    alpha=par[1]
    beta=par[2]
    background = pow(x/beta,-1.0*alpha) #(x/beta)^-alpha
    # first peak, Gaussian model
    A1=par[3]
    mu1=par[4]
    sig1=par[5]
    peak1=A1*exp(-0.5*(x-mu1)*(x-mu1)/sig1/sig1) #A*exp((x-mu)^2/2sigma^2)
    # second peak, Gaussian model
    A2=par[6]
    mu2=par[7]
    sig2=par[8]
    peak2=A2*exp(-0.5*(x-mu2)*(x-mu2)/sig2/sig2)
    return bkgScale*background+peak1+peak2 

xrange=(300,1000)
f1 = r.TF1("f1",myfunction,xrange[0],xrange[1],9)  # xrange 300<=x<=1000, there are 9 parameters in this function
f1.SetNpx(500)  # use large number of points in drawing function to resolve small details better
f1.SetParameters(1e9,4.7,40,5000,490,2,1200,800,25)  # define the parameters
f1.SetParNames("BkgScale","alpha","beta","A1","mu1","sig1","A2","mu2","sig2")  # optional, but nice

In [ ]:
tc=r.TCanvas()
f1.Draw()
tc.Draw()

Generate random data according to this distribution

In [ ]:
entries=100000
ranHist1 = r.TH1F("ranHist1", "Random Histogram for my PDF;x;entries",500,xrange[0],xrange[1]);
ranHist1.FillRandom("f1",entries)

In [ ]:
ranHist1.Draw("e")
tc.Draw()

 Now "pretend" that we don't know the paramaters used to generate the data.
 
 All fits begin with initial guesses at the best parameter values

In [ ]:
f1.SetParameters(1e6,4,80,200,550,3,200,700,20)
ranHist1.Draw("e")
f1.Draw("same")
tc.Draw()

To get better qualitative agreement try:

In [ ]:
f1.SetParameters(0.5e6,4,80,200,550,3,200,700,20)


ranHist1.Draw("e")
f1.Draw("same")
tc.Draw()

Try to fit the function to the data:

In [ ]:
result=ranHist1.Fit(f1,"E")
f1=ranHist1.GetFunction("f1")

Notice that there is a problem here. <br>
Look at the result:

In [ ]:
print(f'chi^2: {f1.GetChisquare()}, nDOF: {f1.GetNDF()}, p-value: {f1.GetProb()}')
ranHist1.Draw("e")
f1.Draw("same")
tc.Draw()

Did the fit work? Did it find both peaks? Probably not. In general, you can't count on complex fits to converge without carefully adjusting the starting parameters. Sure, in this example, you could peek at the parameters used to generate the data, but that's not an option in the real world. Go through the process of adjusting the parameters and replotting the function to get a better representation of the data. Then try a fit and see if you can extract the parameters describing the peeks.

In [ ]:
# extract the parameters and their (parabolic) errors
popt = []
perr = []
for i in range(f1.GetNpar()):
    popt.append(f1.GetParameter(i))
    perr.append(f1.GetParError(i))
    print(f'f1.GetParName(i): {popt[i]:10.2f} +- {perr[i]:10.2f}')

For you to try
===
In the file datadist.root you will find a histogram representing data from an unknown distribution.

* Develop your own fitting function/model and see how well you can fit this distribution. 
* You may need to try a variety of functions.
* Include a plot of your best fit at the bottom of this notebook.
* Include your p-value for the best fit and describe how you settled on this fit versus others.
* Show your best fit parameters and their errors
* Plot the fit residuals, eg for each bin plot (fit-data)/data_uncertianty.  For a good fit the points should randomy fluctuate around 0 (eg no large, contiguous regions above or below 0)
* Plot the pull distribution (for a good fit this should be consisten with a normal distribution w/ $\mu=0,\sigma=1$

For this notebook it is assumed that you'll work with ROOT.

In [ ]:
tf=r.TFile("datadist.root")
hist=tf.Get("h")
tc=r.TCanvas()
hist.Draw()
tc.Draw()

In [ ]:
'''
First attempt. Histogram looks like it has 4 bumps at x=2.2, 5.8, 8, and 10.3 so I tried fitting four gaussians
Resulting pvalue is 0.1859 so I feel like I could do a bit better. Next cell is attempt number 2.
'''

f4gaus = r.TF1("f4gaus", "gaus(0) + gaus(3) + gaus(6) + gaus(9)", 0, 12)

params = [
    50, 2.1, 0.5,
    80, 5.8, 0.6,
    100, 8.0, 0.7,
    70, 10.3, 0.6
]

for i, val in enumerate(params):
    f4gaus.SetParameter(i, val)

# Fit quietly within range
fit_result = hist.Fit(f4gaus, "QR")

# Extract chi2, ndf, p-value
chi2 = f4gaus.GetChisquare()
ndf = f4gaus.GetNDF()
pval = r.TMath.Prob(chi2, ndf)

print(f"Chi2 = {chi2:.3f}")
print(f"NDF  = {ndf}")
print(f"Chi2/NDF = {chi2/ndf:.3f}")
print(f"p-value = {pval:.4f}\n")

print("Fit parameters:")
for i in range(f4gaus.GetNpar()):
    par = f4gaus.GetParameter(i)
    err = f4gaus.GetParError(i)
    print(f"  p[{i}] = {par:.4f} ± {err:.4f}")

tc1=r.TCanvas()
hist.Draw("e")
f4gaus.Draw("same")
tc1.Draw()

In [ ]:
'''
Now I tried fitting just three gaussians and seeing if that was better. That fourth peak could be just some fluctuation so I dropped it
pvalue is now 0.2295 so that seems to be better fit. The reduced chi2 is 1.155 which is pretty close to 1. There's no glaring reason why this fit could be wrong
'''
import math
f3g = r.TF1("f3g", "gaus(0) + gaus(3) + gaus(6)", 0, 12)

# Initial parameter guesses: amplitude, mean, sigma for each Gaussian
f3g.SetParameters(60, 2.1, 0.4,    # first peak
                  80, 5.8, 0.5,    # second peak
                  100, 8.0, 0.9)   # third peak

# Perform the fit quietly ("Q") and within range ("R")
fit_result = hist.Fit(f3g, "QR")

# Extract chi2, ndf, and p-value
chi2 = f3g.GetChisquare()
ndf = f3g.GetNDF()
pval = r.TMath.Prob(chi2, ndf)

print(f"Chi2 = {chi2:.3f}")
print(f"NDF  = {ndf}")
print(f"Chi2/NDF = {chi2/ndf:.3f}")
print(f"p-value = {pval:.4f}")

print("\nFit parameters:")
for i in range(f3g.GetNpar()):
    par = f3g.GetParameter(i)
    err = f3g.GetParError(i)
    print(f"  p[{i}] = {par:.4f} ± {err:.4f}")

tc2=r.TCanvas()
hist.Draw("e")
f3g.Draw("same")
tc2.Draw()

In [ ]:
x_vals = []
res_vals = []
err_x = []
err_y = []

for i in range(1, hist.GetNbinsX() + 1):
    x = hist.GetBinCenter(i)
    data = hist.GetBinContent(i)

    # Skip empty bins
    if data == 0:
        continue

    fit = f3g.Eval(x)
    err = math.sqrt(data)

    residual = (fit - data) / err

    x_vals.append(x)
    res_vals.append(residual)
    err_x.append(0)
    err_y.append(0)

# Convert to arrays
import array
x_arr = array.array("d", x_vals)
y_arr = array.array("d", res_vals)
ex_arr = array.array("d", err_x)
ey_arr = array.array("d", err_y)

# --- Create residual plot ---
g_resid = r.TGraphErrors(len(x_arr), x_arr, y_arr, ex_arr, ey_arr)
g_resid.SetTitle("Fit Residuals;Bin center;x = (fit - data)/#sigma")
g_resid.SetMarkerStyle(20)
g_resid.SetMarkerSize(0.8)

# Draw
c_resid = r.TCanvas("c_resid", "Residuals", 800, 400)
g_resid.Draw("AP")

# Add zero line for visual reference
line = r.TLine(hist.GetXaxis().GetXmin(), 0, hist.GetXaxis().GetXmax(), 0)
line.SetLineColor(r.kRed)
line.SetLineStyle(2)
line.Draw("same")

c_resid.Draw()


In [ ]:

import math

# --- Create pull histogram ---
hpull = r.TH1F("hpull", "Pull Distribution;Pull;Entries", 40, -5, 5)

# Loop over bins of the data histogram
for i in range(1, hist.GetNbinsX() + 1):
    x = hist.GetBinCenter(i)
    data = hist.GetBinContent(i)

    # Skip empty bins
    if data <= 0:
        continue

    fit = f3g.Eval(x)
    err = math.sqrt(data)
    if err == 0:
        continue

    # Compute pull (data - fit)/σ
    pull = (data - fit) / err

    # Fill histogram
    if math.isfinite(pull):
        hpull.Fill(pull)

# --- Fit pull distribution with Gaussian ---
pull_fit = r.TF1("pull_fit", "gaus", -5, 5)
hpull.Fit(pull_fit, "Q")  # Quiet fit

# --- Draw pull histogram ---
c_pull = r.TCanvas("c_pull", "Pull Histogram", 600, 400)
hpull.SetLineColor(r.kBlue + 1)
hpull.SetFillColorAlpha(r.kBlue - 9, 0.4)
hpull.Draw("HIST")

# Draw Gaussian fit on top
pull_fit.SetLineColor(r.kRed)
pull_fit.SetLineWidth(2)
pull_fit.Draw("same")

# Optional: show stats and fit results
r.gStyle.SetOptStat(1110)
r.gStyle.SetOptFit(1111)
c_pull.Update()

# --- Print fit results ---
mu = pull_fit.GetParameter(1)
sigma = pull_fit.GetParameter(2)
print(f"Pull distribution fit: mu = {mu:.3f}, sigma = {sigma:.3f}")


In [ ]:
total_data = hist.Integral()
total_fit = sum(f3g.Eval(hist.GetBinCenter(i)) * hist.GetBinWidth(i) 
                for i in range(1, hist.GetNbinsX()+1))
print("Data integral:", total_data, "Fit integral:", total_fit)